In [1]:
import difflib
from pathlib import Path

import matplotlib as plt
import numpy as np
import pandas as pd
from pandas.io.excel import ExcelWriter

## Установка переменных

In [2]:
stocks_xl = Path('data/отчет МЗ.xlsx')
funds_xl = Path('data/отчет ОС.xlsx')
export_file = Path('data/mz_compare.xlsx')
stocks_name_col = 'Номенклатура'
funds_name_col = 'ОС'
books_account = '101.38 "Прочие основные средства – иное движимое имущество учреждения"'
col_counter = 0
precision = 0.5

## Чтение файлов, создание фремов данных

In [3]:
stocks = pd.read_excel(stocks_xl)
funds = pd.read_excel(funds_xl)


In [4]:
# Из фрейма исключаются счета с библиотечным фондом, и задаются новые порядковые номера индексов
funds = funds.loc[funds['Счет'] != books_account]
new_ind = range(funds.shape[0])
funds.set_index(pd.Index(new_ind))
funds.info()

&lt;class &#39;pandas.core.frame.DataFrame&#39;&gt;
Int64Index: 1985 entries, 0 to 1984
Data columns (total 22 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   № п/п                                 1985 non-null   int64  
 1   ОС                                    1985 non-null   object 
 2   Unnamed: 2                            0 non-null      float64
 3   Unnamed: 3                            0 non-null      float64
 4   Unnamed: 4                            0 non-null      float64
 5   Инвентарный номер                     1723 non-null   object 
 6   ОКОФ                                  1969 non-null   object 
 7   Амортизационная группа                1924 non-null   float64
 8   Способ начисления амортизации         1684 non-null   object 
 9   Дата принятия к учету                 1983 non-null   object 
 10  Состояние                             1985 non-null   object 
 11  Мес

In [5]:
funds.loc[0, 'ОС']

&#39;Квартира трехкомнатная&#39;

In [6]:
stocks.loc[[0,1], stocks_name_col]

0          &quot;Индибак&quot; дезинфицирующее средство. 5 л
1    Дезинфицирующее средство &quot;Акватабс&quot; 1,67 №320
Name: Номенклатура, dtype: object

In [7]:
stocks_size, funds_size  = stocks.shape[0], funds.shape[0]
stocks_size, funds_size

(3557, 1985)

In [25]:
matrix = pd.DataFrame(columns=range(stocks_size), index=range(funds_size))
matrix

,0,1,2,3,4,5,6,7,8,9,...,3547,3548,3549,3550,3551,3552,3553,3554,3555,3556
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1981,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1982,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Расчет вероятности совпадений наименований

### Функции подсчета процента совпдаения

In [31]:
from sub_string import main

def foo_3(y, row_num, **kwargs):
    global col_counter
    weight = main(funds.loc[row_num,funds_name_col], stocks.loc[col_counter, stocks_name_col])
    col_counter += 1
    return weight

In [32]:
def foo_2(y, row_num, **kwargs):
    '''Перебор фрейма по колонкам, расчет и запись веса в фрейм'''
    global col_counter
    weight = difflib.SequenceMatcher(None, funds.loc[row_num,funds_name_col], stocks.loc[col_counter, stocks_name_col]).ratio()
    col_counter += 1
    return weight

In [33]:
def foo(x, **kwargs):
    '''Перебор фрейма по строкам, извлечение данных строки'''
    global col_counter
    x = x.apply(foo_3, args=(x.name,))
    col_counter = 0
    return x

### Запуск фуункций подсчета весов

In [34]:
import time

start = time.time()

matrix = matrix.apply(foo, axis=1)

end = time.time()
period = end - start
print(period)

AttributeError: &#39;function&#39; object has no attribute &#39;apply&#39;

In [28]:
#matrix_weight = matrix[matrix[:] > precision]
#matrix_weight = matrix_weight.dropna(how='all')
#matrix_weight

,0,1,2,3,4,5,6,7,8,9,...,3547,3548,3549,3550,3551,3552,3553,3554,3555,3556
12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Сохранени и чтение результатов

In [11]:
matrix.to_csv('data/mz_weights_2.csv', sep=';', index_label='ind')

In [11]:
matrix = pd.read_csv('data/mz_weights_2.csv', sep=';', index_col='ind')
matrix.head(2)

,0,1,2,3,4,5,6,7,8,9,...,3547,3548,3549,3550,3551,3552,3553,3554,3555,3556
ind,,,,,,,,,,,,,,,,,,,,,
0,0.229508,0.149254,0.262295,0.169014,0.195122,0.1,0.148148,0.195122,0.212766,0.212766,...,0.144928,0.2,0.130435,0.375000,0.244898,0.156863,0.148148,0.12,0.333333,0.170732
1,0.229508,0.149254,0.229508,0.169014,0.195122,0.1,0.148148,0.195122,0.255319,0.255319,...,0.144928,0.2,0.130435,0.291667,0.244898,0.156863,0.148148,0.12,0.333333,0.170732


## Получение результатов

### Получение фрейма где указаны веса превышающие установленную точность

In [12]:
matrix_weight = matrix[matrix[:] > precision]
matrix_weight = matrix_weight.dropna(how='all')
matrix_weight

,0,1,2,3,4,5,6,7,8,9,...,3547,3548,3549,3550,3551,3552,3553,3554,3555,3556
ind,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1981,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Получение фрейма где указаны номера колонок превышающие установленную точность

In [234]:
matrix_col = pd.DataFrame(np.where(matrix > precision, matrix.columns, np.nan))
matrix_col = matrix_col.dropna(how='all')
matrix_col = matrix_col.dropna(axis=1, how='all')
matrix_col

,7,59,62,63,64,65,66,67,68,69,...,3511,3512,3513,3514,3515,3518,3524,3525,3528,3545
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1981,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1982,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [244]:
matrix_col.T.count().max()

30

In [246]:
ind_list = ['ОС', '']
ind_list.extend(range(30))
new_df = pd.DataFrame(index=ind_list)

In [247]:
def write_excel(row_to_excel, num):
    #print(f'{num} start excel writer')
    col_list = ['ОС', '']
    col_list.extend(range(len(row_to_excel)-2))    
    mur = pd.DataFrame(row_to_excel, index=col_list, columns=[num])
    #with ExcelWriter(export_file, mode='a') as writer:
        #mur.to_excel(writer, sheet_name=f'лист {num}')
    return mur

In [248]:
def extract_col(x, n_df):
    global new_df    
    row_to_excel = [funds.loc[x.name, funds_name_col], '']
    stocs_list =  stocks.loc[x.dropna().astype('int32').to_list(), stocks_name_col]
    row_to_excel.extend(stocs_list)    
    new_row = write_excel(row_to_excel, x.name)
    new_df[x.name] = new_row
    return new_row

In [249]:
import time


with ExcelWriter(export_file, mode='w') as writer:
        new_df.to_excel(writer, sheet_name='лист')

start = time.time()

matrix_col.apply(extract_col, axis=1, args=(new_df,))

end = time.time()
period = end - start
print(period)

1.2870733737945557


In [231]:
#new_df.to_excel('data/fg.xlsx')

In [250]:
new_df

,0,1,2,3,4,5,6,7,8,9,...,1972,1973,1974,1977,1978,1979,1980,1981,1982,1983
ОС,Квартира трехкомнатная,Квартира двухкомнатная,Квартира трехкомнатная,Квартира двухкомнатная,Квартира однокомнатная,Квартира трехкомнатная,Квартира двухкомнатная,Квартира трехкомнатная,Квартира трехкомнатная,Квартира двухкомнатная,...,Скамья складная,Стойка унифицированная,Столик инструментальный,Палатка лагерная о/с,Палатка лагерная офицерская,Палатка лагерная офицерская,Палатка лагерная офицерская,Стеллаж металлический 3-х ярусный,Стеллаж металлический 3-х ярусный,Стеллаж металлический 3-х ярусный
,,,,,,,,,,,...,,,,,,,,,,
0,Кирка матыга,Кирка матыга,Кирка матыга,Кирка матыга,Кирка матыга,Кирка матыга,Кирка матыга,Кирка матыга,Кирка матыга,Кирка матыга,...,Стакан для сока,Воронка эмалированная,Инструмент сапожный,Вилка нагрузочная ВН-1,Гладильная доска,Гладильная доска,Гладильная доска,Катетер уретральный металлический женский изог...,Катетер уретральный металлический женский изог...,Катетер уретральный металлический женский изог...
1,Куртка демисезонная,Корзина д\бумаг Феста,Куртка демисезонная,Корзина д\бумаг Феста,Лопатка монтажная,Куртка демисезонная,Корзина д\бумаг Феста,Куртка демисезонная,Куртка демисезонная,Корзина д\бумаг Феста,...,Сумка сержанта,Кружка эмалированная,Набор инструмента AUTO-131,Кислота серная,Каска шахтерская,Каска шахтерская,Каска шахтерская,Линейка металлическая 500 мм,Линейка металлическая 500 мм,Линейка металлическая 500 мм
2,Куртка демисезонная,Куртка демисезонная,Куртка демисезонная,Куртка демисезонная,NaN,Куртка демисезонная,Куртка демисезонная,Куртка демисезонная,Куртка демисезонная,Куртка демисезонная,...,Щетка стальная,Футболка камуфлированная,Набор инструмента AUTO-90,Ложка чайная из н/с,Шапка утепленная полевая,Шапка утепленная полевая,Шапка утепленная полевая,NaN,NaN,NaN
3,NaN,Куртка демисезонная,NaN,Куртка демисезонная,NaN,NaN,Куртка демисезонная,NaN,NaN,Куртка демисезонная,...,NaN,Футболка повседневная,Набор инструмента М-224,Лопата пожарная,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Лопатка монтажная,NaN,Лопатка монтажная,NaN,NaN,Лопатка монтажная,NaN,NaN,Лопатка монтажная,...,NaN,Шайба М6 оцинкованная,Набор инструмента универсальный,Лопата сапёрная,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,Матрац ватный,NaN,Матрац ватный,NaN,NaN,Матрац ватный,NaN,NaN,Матрац ватный,...,NaN,Щетка полировочная,Светильник настольный,Лопата сапёрная,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Тройник соединительный 25,Лопата саперная ББСЛ110,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Тройник соединительный 32,Папка для благодарностей,NaN,NaN,NaN,NaN,NaN,NaN


In [253]:
new_df.T.drop_duplicates().T.to_excel('data/fg1.xlsx')

# Альтернативные решения

In [256]:
difflib.SequenceMatcher(None, 'Палатка каркас.(ПК- 10)', 'Палатка каркасная ПК -10  камуфлированной расцветки').ratio()

0.5135135135135135

In [29]:
difflib.get_close_matches('Палатка каркас.(ПК- 10)', 'Палатка каркасная ПК -10  камуфлированной расцветки')

[]

In [9]:
import re
import string

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import pandas as pd

In [10]:


vectorizer = TfidfVectorizer()
corpus = ['Палатка каркас.(ПК- 10)', 'Палатка каркасная ПК -10  камуфлированной расцветки', 'Палатка']
X = vectorizer.fit_transform(corpus)
feature_names = vectorizer.get_feature_names()
corpus_index = [n for n in corpus]
df = pd.DataFrame(X.T.todense(), index = feature_names, columns = corpus_index)
df.style

,Палатка каркас.(ПК- 10),Палатка каркасная ПК -10 камуфлированной расцветки,Палатка
10,0.480458,0.358291,0.000000
камуфлированной,0.000000,0.471110,0.000000
каркас,0.631745,0.000000,0.000000
каркасная,0.000000,0.471110,0.000000
палатка,0.373119,0.278245,1.000000
пк,0.480458,0.358291,0.000000
расцветки,0.000000,0.471110,0.000000


In [6]:
corpus_1 = funds[funds_name_col].to_list()
corpus_1.extend(stocks[stocks_name_col].to_list())

In [11]:
import time
start = time.time()

tfidf = TfidfVectorizer().fit_transform(corpus)
# no need to normalize, since Vectorizer will return normalized tf-idf
pairwise_similarity = tfidf * tfidf.T
df = pd.DataFrame(pairwise_similarity.todense())
end = time.time()
period = end - start
print(period)




0.004000425338745117


In [12]:
df

,0,1,2
0,1.000000,0.448107,0.373119
1,0.448107,1.000000,0.278245
2,0.373119,0.278245,1.000000


In [8]:
df1 = df[df[:] > precision]

In [24]:
df1[df1.count() > 1]

MemoryError: Unable to allocate 144. MiB for an array with shape (3411, 5542) and data type float64

In [19]:
words = []
for num in df1[5538].dropna().index.to_list():
    words.append(corpus_1[num])
words

[&#39;Адреналин 1 мг/мл 1 мл&#39;,
 &#39;Анальгин 500 мг/мл 2 мл&#39;,
 &#39;Дибазол 10 мг/мл 5 мл&#39;,
 &#39;Диклофенак 25 мг/мл 3 мл&#39;,
 &#39;Кордиамин 250 мг/мл 2 мл&#39;,
 &#39;Преднизолон 30 мг/мл 1 мл&#39;,
 &#39;Супрастин 20 мг/мл 1 мл&#39;,
 &#39;Фуросемид 10 мг/мл(1%) 2 мл&#39;,
 &#39;Цианокобаламин 0,5 мг/мл 1 мл&#39;,
 &#39;Этамзилат 125 мг/мл (12,5%) 2 мл&#39;]

In [13]:
my_text = """Палатка каркас.(ПК- 10)
Палатка каркасная ПК -10  камуфлированной расцветки
""".split('\n')
my_text

[&#39;Палатка каркас.(ПК- 10)&#39;,
 &#39;Палатка каркасная ПК -10  камуфлированной расцветки&#39;,
 &#39;&#39;]

In [14]:
def preprocessing(line):
    line = line.lower()
    line = re.sub(r'[{}]'.format(string.punctuation), ' ', line)
    return line

In [17]:
tfidf_vectorizer = TfidfVectorizer(preprocessor=preprocessing)
tfidf = tfidf_vectorizer.fit_transform(my_text)

kmeans = KMeans(n_clusters=2).fit(tfidf)

In [18]:
kmeans.predict(tfidf_vectorizer.transform(corpus))

array([0, 1])

In [19]:
tfidf_vectorizer.transform(corpus)

&lt;2x7 sparse matrix of type &#39;&lt;class &#39;numpy.float64&#39;&gt;&#39;
	with 10 stored elements in Compressed Sparse Row format&gt;

In [6]:
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('all') # if necessary...


stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize)

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]


print(cosine_sim('Палатка каркас.(ПК- 10)', 'Палатка каркасная ПК -10  камуфлированной расцветки'))
print(cosine_sim('a little bird', 'a little bird chirps'))
print(cosine_sim('a little bird', 'a big dog barks'))

0.3032160644503863
0.7765145304745156
0.17077611319011649
[nltk_data] Error loading all: &lt;urlopen error [Errno 11004]
[nltk_data]     getaddrinfo failed&gt;


In [ ]:
import spacy
nlp = spacy.load('ru_core_news_sm')
doc1 = nlp(u'Hello hi there!')
doc2 = nlp(u'Hello hi there!')
doc3 = nlp(u'Hey whatsup?')

print (doc1.similarity(doc2)) # 0.999999954642
print (doc2.similarity(doc3)) # 0.699032527716
print (doc1.similarity(doc3)) # 0.699032527716